In [36]:
%reset
import numpy as np
from osgeo import gdal, osr, ogr
from skimage import exposure
from skimage.segmentation import quickshift, slic
from sklearn.ensemble import RandomForestClassifier
import time
import os
import matplotlib as mp
import matplotlib.pyplot as plt
import scipy
from scipy import stats


#naip_fn = 'C:/Users/st1154414/Desktop/VP2/Data/Sursee_2020/2020-10-22_adi_Wetzwil-Mspec/img/MSP_00251_00005.tif'#
#naip_fn = "C:/Users/st1154414/Desktop/VP2/Data/Sursee_2020/2020-10-22_adi_beromuenster-nir/img/IMG_5971.JPG"
#naip_fn = "C:/Users/st1154414/Desktop/VP2/Data/Sursee_2020/2020-10-22_adi_Wetzwil-RGB/img/DSC01500.JPG"
#naip_fn = "C:/Users/st1154414/Desktop/VP2/Processed/Orthophoto_Berom_rgb.tif"
#naip_fn = "C:/Users/st1154414/Desktop/VP2/Processed/merged2.tif"
#naip_fn = "D:/cbr/VP2/CodeOutput/output_file_mspec.tif"
naip_fn = "D:/cbr/VP2/VP2_SoilMapping/CodeOutput/output_file_channelFusion.tif"

print(naip_fn)
driverTiff = gdal.GetDriverByName('GTiff')
naip_ds = gdal.Open(naip_fn)

nbands = naip_ds.RasterCount
band_data = []

print('bands', naip_ds.RasterCount, 'rows', naip_ds.RasterYSize, 'columns', naip_ds.RasterXSize)
for i in range(1, nbands+1):
        band = naip_ds.GetRasterBand(i).ReadAsArray()
        band_data.append(band)

band_data = np.dstack(band_data)
print(band_data.shape)



D:/cbr/VP2/CodeOutput/output_file_channelFusion.tif
bands 12 rows 6442 columns 7126
(6442, 7126, 12)


In [37]:
# scale image values from 0.0 - 1.0
#band_data[np.isnan(band_data)] = 0
band_data[np.isnan(band_data)] = -9999.0
img = exposure.rescale_intensity(band_data, in_range= (0.0, 225))

# do segmentation multiple options with quickshift and slic
seg_start = time.time()
# segments = quickshift(img, convert2lab=False)
#segments = quickshift(img, ratio=0.8, convert2lab=False)
# segments = quickshift(img, ratio=0.99, max_dist=5, convert2lab=False)
#segments = slic(img, n_segments=100000, compactness=0.1, start_label = 1)
#segments = slic(img, n_segments=500000, compactness=0.01, start_label = 1)
segments = slic(img, n_segments=50000, compactness=1, start_label = 1)
print('segments complete', time.time() - seg_start)

def segment_features(segment_pixels):
    features = []
    npixels, nbands = segment_pixels.shape
    for b in range(nbands):
        stats = scipy.stats.describe(segment_pixels[:, b])
        band_stats = list(stats.minmax) + list(stats)[2:]
        if npixels == 1:
            band_stats[3] = 0.0
        features += band_stats
    return features

segments complete 114.05887508392334


In [38]:
segment_ids = np.unique(segments)
objects = []
object_ids = []
for id in segment_ids:
    segment_pixels = img[segments == id]
    object_features = segment_features(segment_pixels)
    objects.append(object_features)
    object_ids.append(id)

In [39]:
train_fn = "D:/cbr/VP2/VP2_SoilMapping/Shapes/train.shp"
train_ds = ogr.Open(train_fn)
lyr = train_ds.GetLayer()
driver = gdal.GetDriverByName('MEM')
target_ds = driver.Create('', naip_ds.RasterXSize, naip_ds.RasterYSize, 1, gdal.GDT_UInt16)
target_ds.SetGeoTransform(naip_ds.GetGeoTransform())
target_ds.SetProjection(naip_ds.GetProjection())
options = ['ATTRIBUTE=id']
gdal.RasterizeLayer(target_ds, [1], lyr, options=options)
data = target_ds.GetRasterBand(1).ReadAsArray()
print('min', data.min(), 'max', data.max(), 'mean', data.mean())

ground_truth = target_ds.GetRasterBand(1).ReadAsArray()

classes = np.unique(ground_truth)[1:]
print('class values', classes)

segments_per_class = {}
for klass in classes:
    segments_of_class = segments[ground_truth == klass]
    segments_per_class[klass] = set(segments_of_class)
    print("Training segment for class", klass, ":", len(segments_of_class))

intersection = set()
accum = set()

for class_segments in segments_per_class.values():
    intersection |= accum.intersection(class_segments)
    accum |= class_segments

assert len(intersection) == 0, "Segments represent multiple classes"

    


min 0 max 6 mean 3.921082379065323e-06
class values [1 3 4 5 6]
Training segment for class 1 : 17
Training segment for class 3 : 8
Training segment for class 4 : 14
Training segment for class 5 : 7
Training segment for class 6 : 8


In [40]:
train_img = np.copy(segments)
threshold = train_img.max() + 1

for klass in classes:
    class_label = threshold + klass
    for segment_id in segments_per_class[klass]:
        train_img[train_img == segment_id] = class_label

train_img[train_img <= threshold] = 0
train_img[train_img > threshold] -= threshold

training_objects = []
training_labels = []

for klass in classes:
    class_train_object = [v for i,v in enumerate(objects) if segment_ids[i] in segments_per_class[klass]]
    training_labels += [klass] *len(class_train_object)
    training_objects += class_train_object
    print('Training objects for class', klass, ':', len(class_train_object))

classifier = RandomForestClassifier(n_jobs=1)
classifier.fit(training_objects, training_labels)
print('Fitting Random Forest Classifier')
predicted = classifier.predict(objects)
print('Predicting Classification')

clf = np.copy(segments)
for segment_id, klass in zip(segment_ids, predicted):
    clf[clf == segment_id] = klass

print('Prediction applied to numpy array')

mask = np.sum(img, axis=2)
mask[mask> 0] = 1.0
mask[mask == 0] = -1.0

clf = np.multiply(clf, mask)
clf[clf < 0] = -9999.0

print('Saving classification to raster')
def CreateGeoTiff(Name, Array, driver, GeoT, Projection, DataType):
    DataSet = driver.Create(Name, naip_ds.RasterXSize, naip_ds.RasterYSize, 1, DataType)
    DataSet.SetGeoTransform(GeoT)
    DataSet.SetProjection(Projection)
    DataSet.GetRasterBand(1).SetNoDataValue(-9999.0)
    DataSet.GetRasterBand(1).WriteArray(Array)
    DataSet.FlushCache()
    return Name

CreateGeoTiff('D:/cbr/VP2/VP2_SoilMapping/Processed/classified_mspec.tif', clf, gdal.GetDriverByName ( "GTiff" ), naip_ds.GetGeoTransform(), naip_ds.GetProjection(), gdal.GDT_Float32)
print('Done')

    

Training objects for class 1 : 17
Training objects for class 3 : 8
Training objects for class 4 : 14
Training objects for class 5 : 7
Training objects for class 6 : 8
Fitting Random Forest Classifier
Predicting Classification
Prediction applied to numpy array
Saving classification to raster
Done
